In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 4):  # There are 3 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)


        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('rajasthan_bus_details.csv', index=False)

# Close the driver
driver.quit()

Error occurred while accessing page 3: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7C8376CB5+28821]
	(No symbol) [0x00007FF7C82E3840]
	(No symbol) [0x00007FF7C818578A]
	(No symbol) [0x00007FF7C81D91BE]
	(No symbol) [0x00007FF7C81D94AC]
	(No symbol) [0x00007FF7C8222647]
	(No symbol) [0x00007FF7C81FF33F]
	(No symbol) [0x00007FF7C821F412]
	(No symbol) [0x00007FF7C81FF0A3]
	(No symbol) [0x00007FF7C81CA778]
	(No symbol) [0x00007FF7C81CB8E1]
	GetHandleVerifier [0x00007FF7C86AFCAD+3408013]
	GetHandleVerifier [0x00007FF7C86C741F+3504127]
	GetHandleVerifier [0x00007FF7C86BB5FD+3455453]
	GetHandleVerifier [0x00007FF7C843BDBB+835995]
	(No symbol) [0x00007FF7C82EEB5F]
	(No symbol) [0x00007FF7C82EA814]
	(No symbol) [0x00007FF7C82EA9AD]
	(No symbol) [0x00007FF7C82DA199]
	BaseThreadInitThunk [0x00007FFFBA46257D+29]
	RtlUserThreadStart [0x00007FFFBB40AF08+40]



NameError: name 'driver' is not defined

In [2]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Jodhpur to Ajmer,https://www.redbus.in/bus-tickets/jodhpur-to-a...,RSRTC - 192566,Semi Deluxe Non AC Seater 2+2,04:45,04h 30m,09:15,4.0,INR 250,35 Seats available
1,Jodhpur to Ajmer,https://www.redbus.in/bus-tickets/jodhpur-to-a...,RSRTC - 146314,Super Luxury Volvo AC Seater Pushback 2+2,06:15,04h 00m,10:15,3.9,INR 460,17 Seats available
2,Jodhpur to Ajmer,https://www.redbus.in/bus-tickets/jodhpur-to-a...,RSRTC - 185141,Super Luxury Volvo AC Seater Pushback 2+2,07:15,04h 00m,11:15,4.2,INR 460,28 Seats available
3,Jodhpur to Ajmer,https://www.redbus.in/bus-tickets/jodhpur-to-a...,RSRTC - 197630,Express Non AC Seater 2+3,07:40,04h 40m,12:20,3.9,INR 227,42 Seats available
4,Jodhpur to Ajmer,https://www.redbus.in/bus-tickets/jodhpur-to-a...,RSRTC - 193585,Express Non AC Seater 2+3,07:40,04h 40m,12:20,3.9,INR 227,42 Seats available
...,...,...,...,...,...,...,...,...,...,...
1178,Udaipur to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/udaipur-to-j...,Ajay Raj Travel Agency,A/C Seater / Sleeper (2+1),18:30,08h 45m,03:15,2.7,416,34 Seats available
1179,Udaipur to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/udaipur-to-j...,Amit Travels,A/C Seater / Sleeper (2+1),14:00,09h 00m,23:00,1.8,INR 450,32 Seats available
1180,Udaipur to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/udaipur-to-j...,Safar travels and cargo,A/C Sleeper (2+1),10:00,07h 00m,17:00,0,INR 1500,4 Seats available
1181,Udaipur to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/udaipur-to-j...,Gajraj bus service,Bharat Benz A/C Sleeper (2+1),02:55,09h 05m,12:00,0,INR 1400,25 Seats available
